# <font color=darkred> IDSS32102</font>

<h1><center> Sparse Implementation of LSH</center></h1>

Locality Sensitive Hashing (LSH) can be implemented for both sparse and dense vectors. In this notebook we will implement the algorithm for searching sparse vectors. We will be using k-shingling and minhashing to create our sparse vectors before applying them for search with LSH.



# k-shingling:

We start by defining a few sentences.

In [11]:
a = "flying bird found in the space"
b = "he will not allow you to bring your sticks of dynamite and pet armadillo along"
c = "he figured a few sticks of dynamite were easier than a fishing pole to catch an armadillo"

The first thing we do is create our shingles, we will use k-shingles where k == 2. For longer text it is recommended to create shingles where it is unlikely to produce matching shingles between non-matching text, k values of 7 to 11 would likely produce this outcome.


In [12]:
k = 2
for i in range(len(a) - k+1):
    print(a[i: i+k], end='|')


fl|ly|yi|in|ng|g | b|bi|ir|rd|d | f|fo|ou|un|nd|d | i|in|n | t|th|he|e | s|sp|pa|ac|ce|

These are our shingles, however, we must remove duplicate values as we are producing a set. We do this using the Python type set. 

1) Define a shingle function to apply shingling to each of our sentences:


In [13]:
def shingle(text: str, k: int):
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i: i+k])
    return set(shingle_set)

In [14]:
a = shingle(a, k)
b = shingle(b, k)
c = shingle(c, k)
print(a)

{'ir', 'ou', 'rd', ' i', 'th', ' b', 'ly', 'un', 'n ', 'bi', 'yi', 'g ', 'in', 'e ', 'he', ' s', 'nd', ' f', 'pa', 'ac', 'ce', ' t', 'ng', 'sp', 'fo', 'fl', 'd '}


Now that we have our three shingles we create a shingle vocabulary by create a union between all three sets.

In [15]:
vocab = list(a.union(b).union(c))
print(vocab)

['tc', 'ir', 'we', 'fi', ' n', 'di', 'ou', 'rd', ' b', 'ly', 'un', 'rm', 'ow', 'yi', 'ar', 'no', ' d', 'he', 'et', 'at', 'w ', 'as', 'ri', 'lo', 'o ', 'te', 'mi', 'is', ' o', ' s', 'br', ' c', 'ac', ' y', 'am', 'po', 'on', 'wi', 'er', 'ng', 'dy', 'fo', 'fl', 'an', 'ca', 's ', ' a', 't ', 'sh', 'it', 'ti', 'u ', 'si', ' i', ' e', 'th', 'ad', 'ed', 'ur', 'f ', 'st', 'n ', 're', 'bi', 'g ', 'e ', 'in', 'yn', 'gu', 'ie', 'ck', 'to', 'ig', 'l ', 'ot', 'r ', 'pe', 'ol', 'ma', 'le', 'ew', 'nd', 'ks', 'ic', ' p', 'h ', ' f', 'ea', 'hi', 'pa', 'll', 'ce', ' t', 'il', 'fe', ' w', 'na', 'ch', 'al', 'sp', 'a ', 'd ', 'yo', 'ha', 'of']


Using this vocab we can create one-hot encoded sparse vectors to represent our shingles.

In [16]:
a_1hot = [1 if x in a else 0 for x in vocab]
b_1hot = [1 if x in b else 0 for x in vocab]
c_1hot = [1 if x in c else 0 for x in vocab]
print(a_1hot)

[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]


So we now have one-hot encoded sparse vectors we can move onto minhashing.

# Minhashing

Minhashing is the next step in our process. After creating our shingle sets we use minhashing to create signatures from those sets.

To create our minhashing function we build several hash functions, each will randomly count from 1 to len(vocab) + 1 - creating a random vector:


In [17]:
hash_ex = list(range(1, len(vocab)+1))
print(hash_ex)  # we haven't shuffled yet

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105]


In [40]:
from random import shuffle

shuffle(hash_ex)
print(hash_ex)

[71, 95, 91, 32, 92, 2, 101, 6, 35, 29, 76, 41, 83, 38, 98, 47, 16, 57, 26, 72, 99, 104, 75, 103, 67, 96, 84, 60, 48, 49, 7, 86, 19, 74, 44, 14, 43, 53, 87, 62, 52, 11, 68, 17, 97, 31, 20, 21, 45, 66, 55, 70, 94, 33, 80, 79, 77, 39, 4, 3, 9, 105, 89, 22, 1, 37, 82, 34, 78, 36, 40, 85, 56, 24, 50, 25, 93, 59, 65, 69, 90, 102, 61, 58, 28, 10, 23, 5, 12, 73, 54, 30, 15, 88, 8, 13, 100, 81, 51, 64, 27, 18, 63, 42, 46]


In [41]:
hash_ex.index(2)

5

We now have a randomized list of integers which we can use in creating our hashed signatures. What we do now is begin counting from 1 through to len(vocab) + 1, extracting the position of this number in our new hash_ex list, like so (for the first few integers):


In [43]:
for i in range(1, 5):
    print(f"{i} -> {hash_ex.index(i)}")

1 -> 64
2 -> 5
3 -> 59
4 -> 58



What we do with this is count up from 1 to len(vocab) + 1 and find if the resultant hash_ex.index(i) position in our one-hot encoded vectors contains a positive value (1) in that position, like so:


In [45]:
for i in range(1, len(vocab)+1):
    idx = hash_ex.index(i)
    signature_val = a_1hot[idx]
    print(f"{i} -> {idx} -> {signature_val}")
    if signature_val == 1:
        print('match!')
        break

1 -> 64 -> 1
match!


That gives us a first signature of some value. But this is just a single value, and it takes many values to create a signature (100 for example).

So, how to we generate these other values? By using more hash functions! Let's generate a set of hash functions to create a signature vector of length 20.

hash_funcs = []

for _ in range(20)


In [46]:
hash_funcs = []

for _ in range(20):
    hash_ex = list(range(1, len(vocab)+1))
    shuffle(hash_ex)
    hash_funcs.append(hash_ex)

for i in range(3):
    print(f"hash function {i+1}:")
    print(hash_funcs[i])

hash function 1:
[5, 23, 8, 66, 92, 84, 10, 102, 104, 60, 58, 89, 13, 75, 44, 40, 14, 68, 101, 4, 42, 46, 83, 53, 78, 67, 55, 79, 19, 88, 39, 25, 51, 33, 31, 15, 17, 35, 86, 87, 94, 81, 12, 49, 48, 47, 16, 57, 74, 37, 80, 59, 54, 1, 73, 91, 20, 62, 22, 24, 65, 56, 99, 90, 71, 52, 95, 2, 36, 9, 85, 70, 27, 63, 98, 69, 82, 72, 41, 11, 30, 34, 28, 100, 21, 61, 43, 45, 3, 7, 93, 64, 96, 38, 29, 32, 97, 77, 26, 6, 103, 105, 18, 50, 76]
hash function 2:
[42, 93, 54, 76, 62, 53, 14, 32, 57, 51, 102, 23, 13, 99, 77, 34, 43, 24, 3, 8, 90, 21, 78, 58, 103, 28, 31, 82, 64, 63, 18, 39, 81, 33, 26, 30, 91, 101, 80, 12, 48, 70, 89, 65, 83, 41, 7, 59, 56, 19, 25, 75, 60, 73, 45, 96, 87, 36, 61, 22, 2, 17, 5, 44, 79, 92, 46, 66, 16, 15, 95, 40, 67, 88, 1, 97, 10, 84, 100, 4, 29, 50, 38, 98, 85, 71, 72, 9, 11, 105, 104, 27, 68, 86, 47, 74, 6, 49, 69, 52, 55, 35, 20, 94, 37]
hash function 3:
[14, 25, 8, 92, 55, 26, 5, 80, 72, 1, 29, 85, 23, 7, 4, 71, 17, 73, 57, 66, 9, 39, 50, 52, 86, 13, 101, 61, 74, 9

We're only showing the first three hash functions here - we have 20 in total. To create our signatures we simply process each one-hot vector through each hash function, appending the output value to our signature for that vector.

2) Complete the following signature code:


In [57]:
signature = []

for func in hash_funcs:
    for i in range(1, len(vocab)+1):
        idx = func.index(i)
        signature_val = a_1hot[idx]
        print(f"{i} -> {idx} -> {signature_val}")
        if signature_val == 1:
            print('match!')
            break

#print(signature)

1 -> 53 -> 1
match!
1 -> 74 -> 0
2 -> 60 -> 0
3 -> 18 -> 0
4 -> 79 -> 0
5 -> 62 -> 0
6 -> 96 -> 0
7 -> 46 -> 0
8 -> 19 -> 0
9 -> 87 -> 0
10 -> 76 -> 0
11 -> 88 -> 0
12 -> 39 -> 1
match!
1 -> 9 -> 1
match!
1 -> 69 -> 0
2 -> 19 -> 0
3 -> 58 -> 0
4 -> 6 -> 1
match!
1 -> 2 -> 0
2 -> 87 -> 0
3 -> 54 -> 0
4 -> 29 -> 1
match!
1 -> 0 -> 0
2 -> 85 -> 0
3 -> 10 -> 1
match!
1 -> 55 -> 1
match!
1 -> 55 -> 1
match!
1 -> 47 -> 0
2 -> 103 -> 0
3 -> 9 -> 1
match!
1 -> 32 -> 1
match!
1 -> 5 -> 0
2 -> 97 -> 0
3 -> 0 -> 0
4 -> 21 -> 0
5 -> 88 -> 0
6 -> 50 -> 0
7 -> 10 -> 1
match!
1 -> 71 -> 0
2 -> 41 -> 1
match!
1 -> 67 -> 0
2 -> 53 -> 1
match!
1 -> 57 -> 0
2 -> 72 -> 0
3 -> 4 -> 0
4 -> 43 -> 0
5 -> 63 -> 1
match!
1 -> 57 -> 0
2 -> 3 -> 0
3 -> 67 -> 0
4 -> 22 -> 0
5 -> 88 -> 0
6 -> 52 -> 0
7 -> 84 -> 0
8 -> 15 -> 0
9 -> 4 -> 0
10 -> 59 -> 0
11 -> 23 -> 0
12 -> 56 -> 0
13 -> 73 -> 0
14 -> 9 -> 1
match!
1 -> 61 -> 1
match!
1 -> 8 -> 1
match!
1 -> 52 -> 0
2 -> 36 -> 0
3 -> 51 -> 0
4 -> 78 -> 0
5 -> 98 -> 0


In [59]:
hash_funcs[0]

[5,
 23,
 8,
 66,
 92,
 84,
 10,
 102,
 104,
 60,
 58,
 89,
 13,
 75,
 44,
 40,
 14,
 68,
 101,
 4,
 42,
 46,
 83,
 53,
 78,
 67,
 55,
 79,
 19,
 88,
 39,
 25,
 51,
 33,
 31,
 15,
 17,
 35,
 86,
 87,
 94,
 81,
 12,
 49,
 48,
 47,
 16,
 57,
 74,
 37,
 80,
 59,
 54,
 1,
 73,
 91,
 20,
 62,
 22,
 24,
 65,
 56,
 99,
 90,
 71,
 52,
 95,
 2,
 36,
 9,
 85,
 70,
 27,
 63,
 98,
 69,
 82,
 72,
 41,
 11,
 30,
 34,
 28,
 100,
 21,
 61,
 43,
 45,
 3,
 7,
 93,
 64,
 96,
 38,
 29,
 32,
 97,
 77,
 26,
 6,
 103,
 105,
 18,
 50,
 76]

And there we have our minhash produced signature for a. Let's clean up the code and formalize the process a little.


In [51]:

def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(1, size+1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

# we create 20 minhash vectorsx
minhash_func = build_minhash_func(len(vocab), 20)

3) Write a signature function:

In [301]:
def create_hash(vector: list):
    # use this function for creating our signatures (eg the matching)
    minhash_func = build_minhash_func(len(vocab), 20)
    signature = []
    for func in minhash_func:
        a = []
        for i in range(1, len(vector)+1):
            
            idx = func.index(i)
            signature_val = a_1hot[idx]
            print(f"{i} -> {idx} -> {signature_val}")
            if signature_val == 1:
                print('match!')
                a.append(signature_val)
                break
    
        
    return signature

In [265]:
a_sig = create_hash(a_1hot)
b_sig = create_hash(b_1hot)
c_sig = create_hash(c_1hot)

In [302]:
#print(a_sig)
#print(b_sig)

We now have our three minhashed signatures! These signatures, despite being seemingly randomized, will on average have the very similar Jaccard similarity values as our previous sparse vectors. We have reduced the dimensionality of our vectors significantly - but maintained the same information!

4) Write a Jaccard similarity function:


In [303]:
def jaccard(a: set, b: set):
    return #...


a should have lower similarity with b and c:

In [306]:
jaccard(a, b), jaccard(set(a_sig), set(b_sig))

In [307]:
jaccard(a, c), jaccard(set(a_sig), set(c_sig))

In [308]:
jaccard(b, c), jaccard(set(b_sig), set(c_sig))

We're now ready to move onto the LSH process.

## Locality Sensitive Hashing

The approach we will be taking in this notebook is break our signature vector into multiple bands, creating several sub-vectors.

We then hash each of these sub-vectors into a set of buckets, if we find that two sub-vectors from two signature vectors collide (end up in the same hash bucket) we take the two full signature vectors as candidate pairs - which we then compare in full with a similarity metric (like Jaccard similarity, cosine similarity, etc).

There is no 'set' way to hash our signature vectors, and in-fact the simplest approach is to check for equivalence across sub-vectors, which will be our approach.

First, we must define the number of buckets b we would like to create. It's important to note that each bucket must contain an equal number of rows r - and so our signature length must be divisible by b.


In [271]:
def split_vector(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    # code splitting signature in b parts
    subvecs = []
    for i in range(0, len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs

We'll start by splitting into 10 bands, creating rows of 2 - on the small side to be used in a genuine LSH function but good for our example (we'll explore different r and b values soon).

Let's start with our b and c vectors, which should hopefully match in at least one band.

In [309]:
band_b = split_vector(b_sig, 10)
band_b

In [310]:
band_c = split_vector(c_sig, 10)
band_c

Check if they match (we'll rewrite some of this into Numpy soon).

In [311]:
for b_rows, c_rows in zip(band_b, band_c):
    if b_rows == c_rows:
        print(f"Candidate pair: {b_rows} == {c_rows}")
        # we only need one band to match
        break

And let's do the same for a.

In [275]:
band_a = split_vector(a_sig, 10)

In [276]:
for a_rows, b_rows in zip(band_a, band_b):
    if a_rows == b_rows:
        print(f"Candidate pair: {a_rows} == {b_rows}")
        # we only need one band to match
        break

In [277]:
for a_rows, c_rows in zip(band_a, band_c):
    if a_rows == c_rows:
        print(f"Candidate pair: {a_rows} == {c_rows}")
        # we only need one band to match
        break

Okay great, so even with this very simple implementation - we most likely managed to identify sentences b and c as candidate pairs, and identify a as a non-candidate.


## Tuning LSH

We can visualize the probability of returning a candidate pair vs the similarity of the pair for different values of r and b (rows and bands respectively) like so:


In [278]:
def probability(s, r, b):
    # s: similarity
    # r: rows (per band)
    # b: number of bands
    return 1 - (1 - s**r)**b

In [279]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [280]:
results = pd.DataFrame({
    's': [],
    'P': [],
    'r,b': []
})

for s in np.arange(0.01, 1, 0.01):
    total = 100
    for b in [100, 50, 25, 20, 10, 5, 4, 2, 1]:
        r = int(total/b)
        P = probability(s, r, b)
        results = results.append({
            's': s,
            'P': P,
            'r,b': f"{r},{b}"
        }, ignore_index=True)

In [312]:
sns.lineplot(data=results, x='s', y='P', hue='r,b')